In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix, save_npz

file_path = 'cleaned_data_naukri.csv'
chunksize = 1000  
unique_skills = set()

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    chunk['Skills Required'] = chunk['Skills Required'].astype(str).str.split(',')
    unique_skills.update(skill.strip() for sublist in chunk['Skills Required'] for skill in sublist)
unique_skills = sorted(unique_skills)

unique_skills.append('ExtraColumn') 


skill_to_index = {skill: idx for idx, skill in enumerate(unique_skills)}


n_rows = sum(len(chunk) for chunk in pd.read_csv(file_path, chunksize=chunksize))
n_cols = len(unique_skills)
sparse_matrix = lil_matrix((n_rows, n_cols), dtype=np.int8)


row_offset = 0

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    chunk['Skills Required'] = chunk['Skills Required'].astype(str).str.split(',')
    for i, skills_list in enumerate(chunk['Skills Required']):
        for skill in skills_list:
            if skill.strip() in skill_to_index:
                sparse_matrix[row_offset + i, skill_to_index[skill.strip()]] = 1
            else:
                # Handle cases where skill doesn't exist in skill_to_index
                sparse_matrix[row_offset + i, skill_to_index['ExtraColumn']] = 1
                print(f"Skill '{skill.strip()}' not found in skill_to_index. Adding to ExtraColumn.")
    row_offset += len(chunk)


sparse_matrix = sparse_matrix.tocsr()
save_npz('skills_sparse_matrix.npz', sparse_matrix)


pd.Series(unique_skills).to_csv('unique_skills1.csv', index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5408\789480327.py:12: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5408\789480327.py:26: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  n_rows = sum(len(chunk) for chunk in pd.read_csv(file_path, chunksize=chunksize))
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5408\789480327.py:34: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):


In [2]:
import pandas as pd
from scipy.sparse import load_npz
df = pd.read_csv('cleaned_data_naukri.csv')
sparse_matrix = load_npz('skills_sparse_matrix.npz')
unique_skills = pd.read_csv('unique_skills.csv', header=None).squeeze().tolist()
assert sparse_matrix.shape[1] == len(unique_skills), f"Mismatch: {sparse_matrix.shape[1]} columns in matrix vs {len(unique_skills)} unique skills"
skills_df = pd.DataFrame.sparse.from_spmatrix(sparse_matrix, columns=unique_skills)
result_df = pd.concat([df, skills_df], axis=1)
result_df.to_csv('transformed_jobs_skills.csv', index=False)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5408\3394139817.py:5: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('transformed_jobs_skills.csv')


AssertionError: Mismatch: 890 columns in matrix vs 38911 unique skills